In [0]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt

In [0]:
data=pd.read_csv('E:\data\eydata\data_train.csv')

In [0]:
#删除第一列
data.drop(['Unnamed: 0'],inplace=True,axis=1)

In [0]:
#时间戳转换
data.loc[:,'time_entry']=pd.to_datetime(data['time_entry'])
data.loc[:,'time_exit']=pd.to_datetime(data['time_exit'])

In [0]:
#坐标缩放
data['x_entry']=data['x_entry']*(1e-5)
data['y_entry']=data['y_entry']*(1e-5)
data['x_exit']=data['x_exit']*(1e-5)
data['y_exit']=data['y_exit']*(1e-5)

In [0]:
#f1x将设备终点x坐标提出
def f1x(df):
    return df.iloc[-1,9]
#f1y将设备终点y坐标提出
def f1y(df):
    return df.iloc[-1,10]
data_fx=data.groupby('hash').apply(f1x).to_frame()
data_fx.columns=['endx']
data_fy=data.groupby('hash').apply(f1y).to_frame()
data_fy.columns=['endy']

In [0]:
data1=pd.merge(data,data_fx,on='hash')

In [0]:
data2=pd.merge(data1,data_fy,on='hash')

In [0]:
#f2确定终点是否在区域内
def f2x(x):
    return 1 if (x>=3750901.5068*(1e-5) and x<=3770901.5068*(1e-5)) else 0
def f2y(y):
    return 1 if (y>=-19268905.6133*(1e-5) and y<=-19208905.6133*(1e-5)) else 0
data2['targetx']=data2.endx.apply(f2x)
data2['targety']=data2.endy.apply(f2y)
data2['target']=data2.targetx*data2.targety

In [0]:
#f3x将设备最后起点x坐标提出
def f3x(df):
    return df.iloc[-1,7]
#f1y将设备最后起点y坐标提出
def f3y(df):
    return df.iloc[-1,8]
data_px=data.groupby('hash').apply(f3x).to_frame()
data_px.columns=['prex']
data_py=data.groupby('hash').apply(f3y).to_frame()
data_py.columns=['prey']

In [0]:
#f4eentry将设备最后起点时间提出
def f4entry(df):
    return df.iloc[-1,2]
#f1y将设备最后起点y坐标提出
def f4end(df):
    return df.iloc[-1,3]
data_pt=data.groupby('hash').apply(f4entry).to_frame()
data_pt.columns=['time_pre']
data_ft=data.groupby('hash').apply(f4end).to_frame()
data_ft.columns=['time_end']

In [0]:
data2.drop(['vmax','vmin','vmean'],inplace=True,axis=1)

In [0]:
#之前忘记处理，因此在data2的基础上处理
data2_0=pd.merge(data2,data_pt,on='hash')
data2_1=pd.merge(data2_0,data_ft,on='hash')

In [0]:
data3=pd.merge(data2_1,data_px,on='hash')
data4=pd.merge(data3,data_py,on='hash')

In [0]:
#所有最后起点
x3=data4['prex'].values
y3=data4['prey'].values

In [0]:
#边界，中心
xmin=3750901.5068*(1e-5)
xmax=3770901.5068*(1e-5)
ymin=-19268905.6133*(1e-5)
ymax=-19208905.6133*(1e-5)
xcenter=(xmin+xmax)/2
ycenter=(ymin+ymax)/2

In [0]:
# #最小的记录的条数为1
# data2.groupby('hash')['hash'].count().min()
#将记录条数为1的提取出来做特殊处理，即对于只有pre和end的记录，只用pre的信息来预测end
data_count=data4.groupby('hash')['hash'].count().to_frame()
data_count.columns=['counts']
data5=pd.merge(data4,data_count,on='hash')

In [0]:
#entry到exit的时间间隔
data5['time_entry2exit']=(data5.time_exit-data5.time_entry).dt.total_seconds()

In [0]:
#entry到pre的时间间隔
data5['time_entry2pre']=(data5['time_pre']-data5['time_entry']).dt.total_seconds()

In [0]:
#exit时距离pre的时间间隔
data5['time_exit2pre']=(data5['time_pre']-data5['time_exit']).dt.total_seconds()

In [0]:
#exit时距离end的时间间隔,用于计算时间系数1
data5['time_exit2end']=(data5['time_end']-data5['time_exit']).dt.total_seconds()
#时间系数1，exit距离终点时间越近值越大，可信度越高
mintime=pd.to_datetime('00:00:00')
data5['t1']=1-data5['time_exit2end']/(data5['time_end']-mintime).dt.total_seconds()

In [0]:
#pre到end的时间间隔，用于计算时间系数2
data5['time_pre2end']=(data5['time_end']-data5['time_pre']).dt.total_seconds()
#时间系数2，pre距离终点时间越近越大，可信度越高
mintime=pd.to_datetime('00:00:00')
data5['t2']=1-data5['time_pre2end']/(data5['time_end']-mintime).dt.total_seconds()

In [0]:
#继续提取一条轨迹的记录
#将只有一条轨迹的记录提出
d1=data5[data5['counts']==1]
data_1trace=d1[['trajectory_id','time_pre2end','prex','prey','endx','endy','targetx','targety','target']]
data_1trace['target_prex']=data5['prex'].apply(f2x)
data_1trace['target_prey']=data5['prey'].apply(f2y)
data_1trace.rename(columns={'trajectory_id':'hashid'},inplace=True)
data_1trace=data_1trace[['hashid','time_pre2end','prex','prey','target_prex','target_prey','endx','endy','targetx','targety','target']]
data_1trace.to_csv('E:/data/eydata/train_1trace.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
D:\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [0]:
#entry到exit这段位移的平均速度
data5['vx_entry2exit']=(data5['x_exit']-data5['x_entry'])/data5['time_entry2exit']
data5['vy_entry2exit']=(data5['y_exit']-data5['y_entry'])/data5['time_entry2exit']

In [0]:
#exit到pre的平均速度
data5['vx_exit2pre']=(data5['prex']-data5['x_exit'])/(data5['time_exit2pre'])
data5['vy_exit2pre']=(data5['prey']-data5['y_exit'])/(data5['time_exit2pre'])

In [0]:
#因为当entry到exit没有动，时间为0时计算得到的速度会无限大，因此统统修改为0
data5.fillna(0,inplace=True)
data5.replace(float('inf'),0,inplace=True)
data5.replace(float('-inf'),0,inplace=True)

In [0]:
#entry到pre的平均速度
data5['vx_entry2pre']=(data5['prex']-data5['x_entry'])/(data5['time_entry2pre'])
data5['vy_entry2pre']=(data5['prex']-data5['x_entry'])/(data5['time_entry2pre'])

In [0]:
#当entry即为pre时，时间为0，速度无限大，之前已经将最后一个记录提出，此时直接将该行删除即可
data5.dropna(axis=0,inplace=True)

In [0]:
#est_x代表用不同速度从pre到end的预计位置
data5['est_x_entry2exit']=data5['time_pre2end']*data5['vx_entry2exit']+data5['prex']
data5['est_y_entry2exit']=data5['time_pre2end']*data5['vy_entry2exit']+data5['prey']
#xcof位置系数，预计位置距离center越近，可能性越高
data5['xcof_entry2exit']=1-abs(data5['est_x_entry2exit']-xcenter)/((xmax-xmin)/2)
data5['ycof_entry2exit']=1-abs(data5['est_y_entry2exit']-ycenter)/((ymax-ymin)/2)

In [0]:
#est_x代表用不同速度从pre到end的预计位置
data5['est_x_exit2pre']=data5['time_pre2end']*data5['vx_exit2pre']+data5['prex']
data5['est_y_exit2pre']=data5['time_pre2end']*data5['vy_exit2pre']+data5['prey']
#xcof位置系数，预计位置距离center越近，可能性越高
data5['xcof_exit2pre']=1-abs(data5['est_x_exit2pre']-xcenter)/((xmax-xmin)/2)
data5['ycof_exit2pre']=1-abs(data5['est_y_exit2pre']-ycenter)/((ymax-ymin)/2)

In [0]:
#est_x代表用不同速度从pre到end的预计位置
data5['est_x_entry2pre']=data5['time_pre2end']*data5['vx_entry2pre']+data5['prex']
data5['est_y_entry2pre']=data5['time_pre2end']*data5['vy_entry2pre']+data5['prey']
#xcof位置系数，预计位置距离center越近，可能性越高
data5['xcof_entry2pre']=1-abs(data5['est_x_entry2pre']-xcenter)/((xmax-xmin)/2)
data5['ycof_entry2pre']=1-abs(data5['est_y_entry2pre']-ycenter)/((ymax-ymin)/2)

In [0]:
#f5将设备最后起点的id提出
def f5(df):
    return df.iloc[-1,1]
#f1y将设备最后起点y坐标提出
data_id=data.groupby('hash').apply(f5).to_frame()
data_id.columns=['hashid']
data6=pd.merge(data5,data_id,on='hash')

In [0]:
data7=data6.drop(['hash','trajectory_id'],axis=1)

In [0]:
#注意有的数据只有一条，因此需要只用pre的情况来预测
data_pre=data7[['hashid','time_pre2end','prex','prey','endx','endy','targetx','targety','target']]
data_pre['target_prex']=data7['prex'].apply(f2x)
data_pre['target_prey']=data7['prey'].apply(f2y)
# data_pre.rename(columns={'trajectory_id':'hashid'},inplace=True)
data_pre=data_pre[['hashid','time_pre2end','prex','prey','target_prex','target_prey','endx','endy','targetx','targety','target']]
data_pre.drop_duplicates(inplace=True)
data_pre.to_csv('E:/data/eydata/train_pre.csv')

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [0]:
l8=['hashid',
 'counts',
 'est_x_entry2exit',
 'est_x_entry2pre',
 'est_x_exit2pre',
 'est_y_entry2exit',
 'est_y_entry2pre',
 'est_y_exit2pre',
 't1',
 't2',
 'time_entry2exit',
 'time_entry2pre',
 'time_exit2end',
 'time_exit2pre',
 'vx_entry2exit',
 'vx_entry2pre',
 'vx_exit2pre',
 'vy_entry2exit',
 'vy_entry2pre',
 'vy_exit2pre',
 'x_entry',
 'x_exit',
 'xcof_entry2exit',
 'xcof_entry2pre',
 'xcof_exit2pre',
 'y_entry',
 'y_exit',
 'ycof_entry2exit',
 'ycof_entry2pre',
 'ycof_exit2pre']

In [0]:
data8=data7[l8]
data8.to_csv('E:/data/eydata/train.csv')